In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:

# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(features):
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    X_selected = X_scaled[:,features]
    error_list=[]
    
    for train_index, val_index in kf.split(X_selected):
        X_train, X_test = X_selected[train_index], X_selected[val_index]
        y_train, y_test = y[train_index], y[val_index]
        svr = SVR(kernel='linear')
        svr.fit(X_train, y_train)
        y_pred = svr.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        error_list.append(mae)
    return 1 / (1 + np.mean(error_list))
#====================================================================================================================

# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj < current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
    
#======================================================================================================================

        
pbounds = {'C': (0.1, 10),
           'epsilon': (0.01, 1),
           'gamma': (0.01, 100),
           'kernel': (0, 3)  # Use indices for categorical parameter
}
kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}

def svr_model(kernel,C, epsilon, gamma):
    model = SVR( kernel = kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.
    
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=svr_model,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )


    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
    print("Best Hyperparameters:")
    print(best_params)

    best_C = best_params['C']
    best_epsilon = best_params['epsilon']
    best_gamma = best_params['gamma']
    best_kernel= kernel_mapping[int(best_params['kernel'])]

    # Train the final SVR model with the best parameters
    final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma,kernel= best_kernel)
    final_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
    
print(f"Mean Absolute Error: {np.mean(mean_MAE)}") 
print(f"Mean Magnitude of Relative Error (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE): {np.mean(mean_RMSE)}")


[1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1]
|   iter    |  target   |     C     |  epsilon  |   gamma   |  kernel   |
-------------------------------------------------------------------------
| 1         | -1.754e+0 | 3.808     | 0.9512    | 73.2      | 1.796     |
| 2         | -1.796e+0 | 1.645     | 0.1644    | 5.818     | 2.599     |
| 3         | -1.796e+0 | 6.051     | 0.711     | 2.068     | 2.91      |
| 4         | -1.749e+0 | 8.341     | 0.2202    | 18.19     | 0.5502    |
| 5         | -1.754e+0 | 3.112     | 0.5295    | 43.2      | 0.8737    |
| 6         | -1.753e+0 | 6.157     | 0.1481    | 29.22     | 1.099     |
| 7         | -1.754e+0 | 4.615     | 0.7873    | 19.98     | 1.543     |
| 8         | -1.751e+0 | 5.965     | 0.05599   | 60.76     | 0.5116    |
| 9         | -1.921e+0 | 0.744     | 0.9494    | 96.56     | 2.425     |
| 10        | -1.754e+0 | 3.116     | 0.1067    | 68.43     | 1.32      |
| 11        | -1.751e+0 | 10.0      | 1.0       | 48.1    

| 24        | -1.06e+04 | 10.0      | 0.01      | 76.09     | 0.0       |
| 25        | -1.06e+04 | 10.0      | 1.0       | 35.34     | 0.0       |
| 26        | -1.056e+0 | 10.0      | 1.0       | 55.65     | 3.0       |
| 27        | -9.946e+0 | 9.922     | 0.8572    | 24.34     | 2.727     |
| 28        | -1.056e+0 | 10.0      | 0.01      | 12.18     | 3.0       |
| 29        | -1.06e+04 | 10.0      | 1.0       | 0.01      | 0.0       |
| 30        | -1.056e+0 | 0.2008    | 0.9828    | 76.52     | 0.08079   |
| 31        | -1.03e+04 | 0.711     | 0.952     | 62.61     | 2.84      |
| 32        | -1.06e+04 | 4.688     | 1.0       | 52.63     | 0.0       |
| 33        | -9.905e+0 | 9.964     | 0.9334    | 62.66     | 2.716     |
| 34        | -1.056e+0 | 0.1       | 1.0       | 31.42     | 0.0       |
| 35        | -1.057e+0 | 5.027     | 1.0       | 35.26     | 3.0       |
| 36        | -1.06e+04 | 9.751     | 0.8374    | 6.949     | 0.1607    |
| 37        | -1.056e+0 | 0.1       | 

| 9         | -4.19e+03 | 0.744     | 0.9494    | 96.56     | 2.425     |
| 10        | -4.285e+0 | 3.116     | 0.1067    | 68.43     | 1.32      |
| 11        | -4.285e+0 | 3.19      | 0.2759    | 43.15     | 1.136     |
| 12        | -4.285e+0 | 0.1       | 0.9564    | 97.08     | 3.0       |
| 13        | -4.285e+0 | 0.8168    | 0.6989    | 96.26     | 1.375     |
| 14        | -5.57e+03 | 1.169     | 0.9806    | 95.56     | 2.966     |
| 15        | -4.285e+0 | 6.27      | 0.1571    | 29.2      | 1.359     |
| 16        | -9.342e+0 | 9.389     | 0.7093    | 45.02     | 2.995     |
| 17        | -4.285e+0 | 1.84      | 1.0       | 99.6      | 1.203     |
| 18        | -4.447e+0 | 7.384     | 0.2743    | 24.29     | 0.4766    |
| 19        | -4.285e+0 | 0.1       | 0.01      | 40.57     | 3.0       |
| 20        | -4.285e+0 | 7.674     | 1.0       | 70.06     | 1.434     |
| 21        | -4.284e+0 | 6.87      | 0.01      | 65.58     | 3.0       |
| 22        | -4.284e+0 | 2.198     | 

| 35        | -1.135e+0 | 0.2738    | 0.4789    | 82.66     | 0.00309   |
| 36        | -1.045e+0 | 4.691     | 0.09103   | 13.3      | 2.935     |
| 37        | -1.131e+0 | 9.707     | 0.01326   | 72.31     | 0.09943   |
| 38        | -1.135e+0 | 4.899     | 0.01      | 100.0     | 3.0       |
| 39        | -1.135e+0 | 10.0      | 0.01      | 94.5      | 3.0       |
| 40        | -1.135e+0 | 0.1       | 0.01      | 73.16     | 0.0       |
Best Hyperparameters:
{'C': 0.1740455505462274, 'epsilon': 0.945917994052146, 'gamma': 77.57436949623542, 'kernel': 2.8299833299873307}
Mean Absolute Error: 5254.881974701174
Mean Magnitude of Relative Error (MMRE): 1.20
Root Mean Squared Error (RMSE): 9059.843780408755
